# Background Tasks

> Background tasks are functions run after returning a response.

Useful for operations where you don't want or need the user waiting for it to finish. Typical scenarios include:

- User setup in complex systems where you can inform the user later in email that their account is complete
- Batch processes that take a significant amount of time where you don't want to keep an HTTP connection alive
- Queries to slow LLMs

Background tasks are an easy-to-use wrapper over Python's asyncio library. They are used to improve user experience, often making apps feel faster to the end user.

## A Simple Background Task Example

Bekow we demonstrate attaching a task to FtResponse by assigning it via the background argument. When the page is visited, it will display 'Simple Background Task Example' almost instantly, while in the terminal it will slowly count upward from 0.

``` {.python filename="main.py" code-line-numbers="true"}
from fasthtml.common import *
from starlette.background import BackgroundTask
from time import time, sleep

app, rt = fast_app()

def counter(loops): # <1>
    """Slowly print integers to the terminal"""
    for i in range(loops):
        print(i)
        sleep(i)

@rt
def index():
    task = BackgroundTask(counter, loops=5)  # <2>
    return FtResponse(Titled('Simple Background Task Example'), background=task) # <3>

serve()
```

1. `counter` is our task function. There is nothing special about it, although it is a good practice for its arguments to be serializable as JSON
2. We use `starlette.background.BackgroundTask` to turn the counter function into a background task
3. `FtResponse` is called explicitly so we can attack the task to its background. Normally we don't need to call `FtResponse` explicitly, setting background tasks is the exception.

::: {.callout-caution}
## Background tasks are not distributed task queues

While background tasks often provides the user with a faster experience, the server itself isn't accelerated. What that means is that processes are still happening, but are hidden. So if a server is struggling under the load of a lot of user activity, so long as it isn't an issue with HTTP, background tasks won't help with server load.

This is where full-fledged distributed task queue libraries like Celery and Dramatiq come into play. At the cost of dramatically increased complexity over background tasks they allow for the distribution of tasks over addition servers, as well as providing improved observability, retry mechanisms, and persistence in case of server shutdown.

In our experience, it's often better to build something with background tasks and then convert it to a task queue.
:::